# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from collections import defaultdict

In [5]:
from tqdm import tqdm

In [6]:
import time, gc

In [7]:
import importlib

In [8]:
from itertools import product


In [9]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [10]:
from util import util_pipeline

In [11]:
from util import util_experiment

In [12]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [13]:
PATH_QUERY = '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

# Data load

## Query data load

In [14]:
df_query = pd.read_csv(PATH_QUERY)

In [15]:
df_query.shape

(16045, 11)

In [16]:
df_query[df_query['ID']==2845]

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
5618,2845,"Em regra, o pregão é a modalidade de licitação adequada para a concessão remunerada de uso de be...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 478/2016 - Plenário,2016-03-02,RELATOR,MARCOS BEMQUERER


In [17]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


## Qrel data load

In [18]:
df_qrel = pd.read_csv(PATH_QREL)

In [19]:
df_qrel.shape

(94809, 3)

In [20]:
df_qrel.head()

,ID_QUERY,ID_DOCTO,TYPE
0,158,15147,INDEXACAO_EXTRA
1,37,15147,INDEXACAO_EXTRA
2,178,15147,INDEXACAO_EXTRA
3,14564,15147,INDEXACAO_EXTRA
4,9219,15147,INDEXACAO_EXTRA


In [21]:
df_qrel.shape[0]

94809

In [22]:
df_search_data = df_query.merge(df_qrel, how='left', left_on='ID', right_on='ID_QUERY').drop('ID_QUERY', axis=1)

In [23]:
df_search_data.shape[0]


94809

In [24]:
# Agrupar os dados pelo ID e criar a coluna 'RELEVANCE_LIST' com a lista de tuplas
# df_new = df_search_data.groupby('ID').apply(lambda x: list(zip(x['ID_DOCTO'], x['TYPE']))).reset_index(name='RELEVANCE_LIST')
# df_new['RELEVANCE_LIST'] = df_new['RELEVANCE_LIST'].apply(lambda x: sorted(x, key=lambda tup: ('AREA', 'TEMA', 'SUBTEMA', 'INDEXACAO_EXTRA').index(tup[1])))
df_new = df_search_data.groupby('ID').apply(lambda x: dict(zip(x['ID_DOCTO'], x['TYPE']))).reset_index(name='RELEVANCE_DICT')


In [25]:
df_new.head()

,ID,RELEVANCE_DICT
0,5,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,6,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,7,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,8,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,9,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [26]:
# Unir o resultado com as demais colunas únicas
df_new = pd.merge(df_new, df_search_data.drop_duplicates('ID'), on='ID', how='left')

In [27]:

# Selecionar as colunas desejadas
df_search_data = df_new[['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE', 'NORMATIVE_AUTHOR_NAME', 'RELEVANCE_DICT']]


In [28]:
df_search_data.shape

(16045, 12)

In [29]:
df_search_data = df_search_data.reset_index()

In [30]:
df_search_data.head()

,index,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,RELEVANCE_DICT
0,0,5,"SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial co...","Lei Ordinária 3738/1960 || Lei Ordinária 1711/1952, art. 184",SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,1,6,"SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fund...",Lei Ordinária 3738/1960,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,2,7,SÚMULA TCU 3: O arquivamento é a solução indicada para as hipóteses em que as contas de responsá...,NaN,SUMULA,Direito processual,5288,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,3,8,"SÚMULA TCU 4: A reclassificação de cargos não aproveita ao servidor aposentado, a menos que lei ...",NaN,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,4,9,"SÚMULA TCU 5 (REVOGADA): As sociedades de economia mista, salvo disposição expressa em lei, não ...",NaN,SUMULA,Competência do TCU,5095,NaN,AC 2082/2007-PL,2007-10-03,RELATOR,UBIRATAN AGUIAR,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [31]:
del df_new

# Create reference to index (elastic search)

Documentation https://docs.haystack.deepset.ai/docs
https://haystack.deepset.ai/tutorials/09_dpr_training 


In [32]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu', 'uuid': 'RD_aAWppQH6vjNxKUdL_0w', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '16045', 'store.size': '334.9mb', 'pri.store.size': '334.9mb'}

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'fGjzBrQzRUWTZ6KlnbfLqw', 'pri': '1', 'rep': '1', 'docs.count': '13255', 'docs.deleted': '2755', 'store.size': '294.9mb', 'pri.store.size': '294.9mb'}



In [33]:
index_dict

{'indir_juris_tcu': {'health': 'yellow',
  'status': 'open',
  'index': 'indir_juris_tcu',
  'uuid': 'RD_aAWppQH6vjNxKUdL_0w',
  'pri': '1',
  'rep': '1',
  'docs.count': '16045',
  'docs.deleted': '16045',
  'store.size': '334.9mb',
  'pri.store.size': '334.9mb'},
 'indir_juris_tcu_index': {'health': 'yellow',
  'status': 'open',
  'index': 'indir_juris_tcu_index',
  'uuid': 'fGjzBrQzRUWTZ6KlnbfLqw',
  'pri': '1',
  'rep': '1',
  'docs.count': '13255',
  'docs.deleted': '2755',
  'store.size': '294.9mb',
  'pri.store.size': '294.9mb'}}

In [34]:
INDEX_NAME = 'indir_juris_tcu_index'

In [35]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13255

Qtd de embeddings 13255

Documento.id=5: <Document: id=5, content='O termo é "Abono de permanência em serviço".
Abono de permanência em serviço tem definição: "O abono...'>


# Pipelines creation

In [36]:
pipes = []

In [37]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou."

In [38]:
raise Exception ('Stop execution - create only desired pipelines in code below ')

Exception: Stop execution - create only desired pipelines in code below 

In [ ]:
importlib.reload(util_pipeline)

<module 'util.util_pipeline' from '/home/borela/fontes/ind-ir/code/util/util_pipeline.py'>

pype_type: it will be the sufix of file name as we can generate train dataset with different pipelines


In [56]:
%%time
pipe = util_pipeline.return_pipeline_join_bm25_sts_reranker(index, 'MONOT5', parm_limit_query_size=350)
pipe_type = 'join_sts_bm25_ranker_monot5_3b'

CPU times: user 39.6 s, sys: 8.93 s, total: 48.5 s
Wall time: 55.6 s


In [57]:
%%time
doc_returned = pipe.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 10},
         "StsRetriever": {"top_k": 10},
         "filters":{"$and": {"count_index_area": {"$gte": 1},
                             "$not": {"id": {"$in": [1131, 342]}}
                            }}})


CPU times: user 829 ms, sys: 247 ms, total: 1.08 s
Wall time: 1.13 s


In [ ]:
util_pipeline.detail_document_found(doc_returned, parm_num_doc=15)

Parâmetros usados: {'Bm25Retriever': {'top_k': 150}, 'StsRetriever': {'top_k': 150}, 'filters': {'$and': {'count_index_area': {'$gte': 1}, '$not': {'id': {'$in': [1131, 342]}}}}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 8
Primeiro docto:
<Document: id=5095, content='O termo é "Competência do TCU". Competência do TCU tem definição: "São as seguintes as competências ...'>

Último (8):
<Document: id=414, content='O termo é "Desestatização".
Desestatização tem definição: "É a retirada da presença do Estado de ati...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Competência do TCU', '5095', -1.4214444160461426]
1 : ['Responsabilidade', '775', -2.1346242427825928]
2 : ['Finanças públicas', '629', -4.974789142608643]
3 : ['Gestão administrativa',

In [39]:
%%time
pipe = util_pipeline.return_pipeline_bm25(index)
pipe_type = 'bm25'

CPU times: user 664 µs, sys: 0 ns, total: 664 µs
Wall time: 662 µs


In [40]:
%%time
doc_returned = pipe.run(query=parm_query, 
 params={"Retriever": {"top_k": 150}, 
         "filters":{"$and": {"count_index_area": {"$gte": 1},
                             "$not": {"id": {"$in": [1131, 342]}}
                            }}})


CPU times: user 1.92 ms, sys: 0 ns, total: 1.92 ms
Wall time: 8.56 ms


In [41]:
util_pipeline.detail_document_found(doc_returned, parm_num_doc=15)

Parâmetros usados: {'Retriever': {'top_k': 150}, 'filters': {'$and': {'count_index_area': {'$gte': 1}, '$not': {'id': {'$in': [1131, 342]}}}}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 8
Primeiro docto:
<Document: id=5095, content='O termo é "Competência do TCU".
Competência do TCU tem definição: "São as seguintes as competências ...'>

Último (8):
<Document: id=775, content='O termo é "Responsabilidade".
Responsabilidade tem definição: "Qualidade de quem responde pelos próp...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Competência do TCU', '5095', 0.981022770821533]
1 : ['Desestatização', '414', 0.9236313236529244]
2 : ['Gestão administrativa', '15273', 0.8770019134030279]
3 : ['Contrato administrativo', '331', 0.8510101027285571]
4 : [

In [ ]:
doc_returned['documents'][0].id,  doc_returned['documents'][0].meta['id'],doc_returned['documents'][0].meta['name'], doc_returned['documents'][0].score

('1102005', 1102005, 'Plano de contratações', 0.9856848798183901)

In [ ]:
doc_returned['documents'][0].meta.keys()

dict_keys(['has_scope_note', 'text_scope_note', 'text_example', 'text_specialization', 'text_synonym', 'has_english_translation', 'has_spanish_translation', 'num_word', 'count_index_subtheme', 'has_specialization', 'count_index_area', 'text_definition', 'has_related_term', 'has_generalization', 'class', 'text_spanish_translation', 'text_related_term', 'has_synonym', 'num_tokens_monot5_3b', 'date_reference', 'has_example', 'has_definition', 'count_index_theme', 'len_text_char_log', 'num_tokens_minilm', 'count_index_total', 'len_text_char', 'text_english_translation', 'text_generalization', 'count_index_extra', 'name'])

# Experiments

In [ ]:
df_search_data.head()

,index,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,RELEVANCE_DICT
0,0,5,"SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial co...","Lei Ordinária 3738/1960 || Lei Ordinária 1711/1952, art. 184",SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,1,6,"SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fund...",Lei Ordinária 3738/1960,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,2,7,SÚMULA TCU 3: O arquivamento é a solução indicada para as hipóteses em que as contas de responsá...,NaN,SUMULA,Direito processual,5288,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,3,8,"SÚMULA TCU 4: A reclassificação de cargos não aproveita ao servidor aposentado, a menos que lei ...",NaN,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,4,9,"SÚMULA TCU 5 (REVOGADA): As sociedades de economia mista, salvo disposição expressa em lei, não ...",NaN,SUMULA,Competência do TCU,5095,NaN,AC 2082/2007-PL,2007-10-03,RELATOR,UBIRATAN AGUIAR,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [ ]:
query_data = df_search_data.iloc[0]

In [ ]:
query_data['TEXT'], query_data['RELEVANCE_DICT']

('SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens previstas no art. 184 da Lei nº 1.711, de 28/10/52.',
 {1298: 'INDEXACAO_EXTRA',
  15340: 'INDEXACAO_EXTRA',
  15961: 'INDEXACAO_EXTRA',
  1131: 'AREA',
  5106: 'SUBTEMA',
  1113: 'TEMA'})

In [ ]:
list(query_data['RELEVANCE_DICT'].keys())

[1298, 15340, 15961, 1131, 5106, 1113]

In [ ]:
%%time
doc_returned = pipe.run(query=query_data['TEXT'], 
 params={"Retriever": {"top_k": 150}, "filters":{"count_index_area": {"$gte": 1}}})


CPU times: user 2.15 ms, sys: 0 ns, total: 2.15 ms
Wall time: 11.8 ms


In [ ]:
%%time
doc_returned = pipe.run(query=parm_query, 
 params={"Retriever": {"top_k": 150}, 
         "filters":{"$and": {"count_index_area": {"$gte": 1},
                             "$not": {"id": {"$in": list(query_data['RELEVANCE_DICT'].keys())}}
                            }}})

CPU times: user 2.46 ms, sys: 0 ns, total: 2.46 ms
Wall time: 7.59 ms


In [ ]:
util_pipeline.detail_document_found(doc_returned, parm_num_doc=15)

Parâmetros usados: {'Retriever': {'top_k': 150}, 'filters': {'$and': {'count_index_area': {'$gte': 1}, '$not': {'id': {'$in': [1298, 15340, 15961, 1131, 5106, 1113]}}}}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 9
Primeiro docto:
<Document: id=5095, content='O termo é "Competência do TCU".
Competência do TCU tem definição: "São as seguintes as competências ...'>

Último (9):
<Document: id=775, content='O termo é "Responsabilidade".
Responsabilidade tem definição: "Qualidade de quem responde pelos próp...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Competência do TCU', '5095', 0.9849794090993254]
1 : ['Desestatização', '414', 0.9400033944545403]
2 : ['Gestão administrativa', '15273', 0.8990803951959326]
3 : ['Convênio', '342', 0.8861608632

# Generate train data

In [44]:
# Function to invert keys and values keeping a list of keys for each value
def invert_dict_with_lists(d):
    inverted_dict = defaultdict(list)
    for k, v in d.items():
        inverted_dict[v].append(k)
    return dict(inverted_dict)

In [45]:
df_search_data['RELEVANCE_DICT_TYPE'] = df_search_data['RELEVANCE_DICT'].apply(invert_dict_with_lists)

Bm25 

        100%|██████████| 16045/16045 [08:42<00:00, 30.70it/s]CPU times: user 2min 25s, sys: 2.83 s, total: 2min 28s
        Wall time: 8min 42s

In [60]:
%%time
max_count_negative = 2
list_train_data = []
for cnt_row, (ndx, row_query) in tqdm(enumerate(df_search_data.iterrows()), mininterval=10, total=df_search_data.shape[0]):
    # print(f"cnt {cnt} query_id {row_query['ID']} ")
    set_not_relevant = set()
    list_not_in_docid = list(row_query['RELEVANCE_DICT'].keys())
    for type_relevance in ['AREA', 'TEMA', 'SUBTEMA', 'INDEXACAO_EXTRA']:
        if type_relevance in row_query['RELEVANCE_DICT_TYPE']: # INDEXACAO_EXTRA pode não existir
            for cnt_doc_in_type, doc_id in enumerate(row_query['RELEVANCE_DICT_TYPE'][type_relevance]):
                relevant_data = (row_query['ID'], doc_id, 1, type_relevance)
                # print(f"     {relevant_data}")
                list_train_data.append(relevant_data)
                # Encontra primeiro encontrado para não relevante    
                if type_relevance == 'AREA':
                    type_field_count = "count_index_area"
                elif type_relevance == 'TEMA':
                    type_field_count = "count_index_theme"
                elif type_relevance == 'SUBTEMA':
                    type_field_count = "count_index_subtheme"
                elif type_relevance == 'INDEXACAO_EXTRA':
                    type_field_count = "count_index_extra"
                else:
                    raise Exception(f"type_relevance {type_relevance} not expected!")     
                doc_returned = pipe.run(query=row_query['TEXT'], 
                                    # params={"Retriever": {"top_k": max_count_negative}, # extra may have 9 values
                                    params={"Bm25Retriever": {"top_k": max_count_negative*5},
                                            "StsRetriever": {"top_k": max_count_negative*5},
                                            "filters":{"$and": {type_field_count: {"$gte": 1},
                                            "$not": {"id": {"$in": list_not_in_docid}}
                                            }}})
                if len(doc_returned['documents']) == 0:
                    raise Exception(f"Not found negative examples!")
                count_negative = 0
                for ndx, docto in enumerate(doc_returned['documents']):
                    if count_negative >= max_count_negative:
                        break   
                    if docto.meta['id'] not in set_not_relevant:
                        count_negative += 1
                        set_not_relevant.add(docto.meta['id'])
                        list_not_in_docid.append(docto.meta['id'])
                        not_relevant_data = (row_query['ID'], docto.meta['id'], 0, type_relevance)
                        # print(f"     {not_relevant_data}")                    
                        list_train_data.append(not_relevant_data)
                if count_negative < max_count_negative:
                    print(f"Less negative {count_negative} than {max_count_negative} for query_id {row_query['ID']} in {type_relevance}")
    # if cnt_row > 2: break



 62%|██████▏   | 9884/16045 [20:14:46<12:28:53,  7.29s/it]

In [ ]:
df = pd.DataFrame(list_train_data, columns= ['QUERY_ID', 'DOC_ID', 'RELEVANCE', 'TYPE_RELEVANCE'])

In [ ]:
df.shape
# bm25 (284376, 4)

(284376, 4)

In [ ]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE
0,5,1131,1,AREA
1,5,5095,0,AREA
2,5,932,0,AREA
3,5,1113,1,TEMA
4,5,1112,0,TEMA


In [ ]:
path_thain =  f'../data/train_juris_tcu_index/train_data_juris_tcu_index_{pipe_type}.csv'

In [ ]:
df.to_csv(path_thain, sep=',', index=False)